<a href="https://colab.research.google.com/github/667029/dat158-ML-assignment2/blob/main/testDAT158ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline model med RandomForestRegressor

In [ ]:
!pip install gradio

In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import gradio as gr

In [26]:
# Last inn datasettet
train_url = "https://raw.githubusercontent.com/667029/dat158-ML-assignment2/refs/heads/main/df_train.csv"
test_url = "https://raw.githubusercontent.com/667029/dat158-ML-assignment2/refs/heads/main/df_test.csv"
train_data = pd.read_csv(train_url)
test_data = pd.read_csv(test_url)

In [27]:
# Fjern dato-kolonnen
train_data = train_data.drop(columns=['date'])
test_data = test_data.drop(columns=['date'])

In [28]:
#print("Treningsdata:")
#print(train_data.head())
#print("\nTestdata:")
#print(test_data.head())

In [29]:
#print("\nManglende verdier i treningsdata:", train_data.isnull().sum())
#print("Manglende verdier i testdata:", test_data.isnull().sum())

In [30]:
# Fyll inn manglende verdier
train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(test_data.mean())

In [31]:
# Definer input- og målvariabler
X_train = train_data.drop('price', axis=1)  # Alle kolonner unntatt 'price'
y_train = train_data['price']               # Målvariabelen 'price'

X_test = test_data.drop('price', axis=1)    # Testsett uten 'price'
y_test = test_data['price']                 # Målvariabelen for test

In [ ]:
# Tren RandomForest-modellen
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [33]:
# Gjør prediksjoner på testsettet
y_pred = model.predict(X_test)

In [ ]:
# Evaluering av modellen
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

#print("\nModellevaluering:")
#print("Mean Absolute Error (MAE):", mae)
#print("Mean Squared Error (MSE):", mse)
#print("Root Mean Squared Error (RMSE):", rmse)
#print("R-squared (R²):", r2)

In [ ]:
# Feature importance for å se hvilke variabler som har størst effekt
importances = model.feature_importances_
feature_names = X_train.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
#print("\nFeature Importance:")
#print(feature_importances)

In [36]:
# Funksjon for prediksjon og evaluering
def predict_price(inputs):
    input_data = pd.DataFrame([inputs], columns=X_test.columns)
    pred_price = model.predict(input_data)[0]
    return round(pred_price, 2)

In [37]:
# Funksjon for evaluering av modellen
def model_evaluation():
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    return {
        "Mean Absolute Error (MAE)": round(mae, 2),
        "Mean Squared Error (MSE)": round(mse, 2),
        "Root Mean Squared Error (RMSE)": round(rmse, 2),
        "R-squared (R²)": round(r2, 2)
    }

In [39]:
# Definer Gradio input- og output-komponenter
inputs = [gr.Number(label=col) for col in X_test.columns]
output = gr.Textbox(label="Predikert boligpris")

In [ ]:
# Lag Gradio interface
interface = gr.Interface(
    fn=predict_price,
    inputs=inputs,
    outputs=output,
    title="Boligpris Prediksjon",
    description="Fyll inn funksjoner for å få en predikert boligpris basert på modellen."
)

In [44]:
# Funksjon for å vise modellevaluering
evaluation_output = gr.JSON(label="Modellevaluering")
evaluation_interface = gr.Interface(
    fn=model_evaluation,
    inputs=[],
    outputs=evaluation_output,
    title="Modellevaluering",
    description="Viser evalueringen av den trente modellen."
)

In [ ]:
# Kjør begge Gradio-appene samtidig
interface.launch()
evaluation_interface.launch()